# Generic Rule Induction Notebook

Continuoulsy refined.

In [1]:
#%store -z acc_list 
#%store -z prec_list 

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [2]:
from config import config_dict
from config import config_dict_imbalanced

# document config order
CONFIG = config_dict_imbalanced["CONFIG2"]
print('Proceed with configuration:', CONFIG["NAME"])

Proceed with configuration: fraud_detection


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import wittgenstein as lw
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
if CONFIG['ALGO'] == 'CORELS':
    from corels import *



# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'], nrows = 10000)
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    
    df[CONFIG['TARGET_LABEL']]= pd.Categorical(df[CONFIG['TARGET_LABEL']]).codes
    #df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)

    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

### Data

In [4]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)

    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

0    284315
1       492
Name: Class, dtype: int64

In [5]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

0    284315
1       492
Name: Class, dtype: int64


### Train, Test Split

In [6]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (199364, 30) (199364,)
Test: (85443, 30) (85443,)


### Reference Performance

In [7]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [8]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=False, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=False) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 199364 entries, 2557 to 121958
Columns: 270 entries, ('Time', '<=', 35006.0) to ('Amount', '<=', 203.74100000000035)
dtypes: int32(270)
memory usage: 206.9 MB


feature             Time                                               \
operation             <=                                                
value      35006.000000   47671.200000   60753.000000   73208.400000    
2557                   1              1              1              1   
247823                 0              0              0              0   
152342                 0              0              0              0   
103385                 0              0              0              1   
8771                   1              1              1              1   

feature                                                                \
operation                                                               
value      84604.500000   120336.600000  132824.100000  145161.000000   
2557                   1              1              1              1   
247823                 0              0              0              0   
152342                 0              1              1              1   
103385                 1              1              1              1   
8771                   1              1              1              1   

feature                              V1  ...            V28         Amount  \
operation                            <=  ...             <=             <=   
value      157621.000000 -1.891903       ...  0.179945       1.000000        
2557                   1              1  ...              1              1   
247823                 1              0  ...              1              0   
152342                 1              0  ...              1              0   
103385                 1              0  ...              1              0   
8771                   1              0  ...              0              0   

feature                                                                \
operation                                                               
value      3.570000       8.910000       12.990000      22.000000       
2557                   1              1              1              1   
247823                 0              0              0              0   
152342                 0              0              0              0   
103385                 1              1              1              1   
8771                   0              0              0              0   

feature                                                                
operation                                                              
value      37.000000      59.900000      100.000000     203.741000     
2557                   1              1              1              1  
247823                 0              0              0              0  
152342                 0              0              1              1  
103385                 1              1              1              1  
8771                   0              1              1              1  

[5 rows x 270 columns]

### Rule Induction

In [9]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    estimator = lw.RIPPER()
    estimator.fit(x_train_bin, y_train,class_feat=CONFIG["TARGET_LABEL"] , pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                     max_card=2, # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for BRCG
Learning DNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Training time: 3.491999387741089


In [10]:
CONFIG['POS_CLASS']

1

### Evaluation

In [11]:
acc_list = []
prec_list = []
%store -r acc_list
%store -r prec_list

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, y_pred))

    acc_list.append(recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    prec_list.append(precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    %store acc_list
    %store prec_list
   
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    


Accuracy: 0.9984082955888721
Balanced accuracy: 0.5
Precision: 0.0
Recall: 0.0
F1 0.0
ConfusionMatrix [[85307     0]
 [  136     0]]
F-2 0.0
Mathhews 0.0
Stored 'acc_list' (list)
Stored 'prec_list' (list)


C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [12]:
y_test.value_counts()

0    85307
1      136
Name: Class, dtype: int64

In [13]:
CONFIG['POS_CLASS']

1

In [14]:
df[CONFIG["TARGET_LABEL"]].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [15]:
#XGboost for Binary Classification
# storing the results in a list
if CONFIG['TYPE'] == 'BINARY' and CONFIG['BASELINE'] == True:
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_train_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train_bin.columns.values]
    x_test_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_test_bin.columns.values]

    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train_bin, y_train)
    preds = xgb_cl.predict(x_test_bin)     
    print('Accuracy:', accuracy_score(y_test, preds))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, preds))
    print('Precision:', precision_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('F-2', fbeta_score(y_test, preds, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, preds))
   

Accuracy: 0.9995786664794073
Balanced accuracy: 0.9043531528686357
Precision: 0.9166666666666666
Recall: 0.8088235294117647
F-2 0.8283132530120482
Mathhews 0.860852880025692


In [16]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    print(estimator.rule_list_to_pretty_string())
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        print(f"Antecedents Length Rule {i}:" , an)
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    print("Rule Length:", len(estimator.ruleset_))

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Number of rules: 0
[]
